In [3]:
import sqlite3


class GradationModule:
    def __init__(self, db_connection):
        self.db_connection = db_connection

    def calculate_cumulative_retain(self, weights):
        """Calculates the cumulative weight retained for each sieve size."""
        cumulative_retain = []
        total = 0
        for weight in weights:
            total += weight
            cumulative_retain.append(total)
        return cumulative_retain

    def calculate_percent_passing(self, cumulative_retain, total_sample_weight):
        """Calculates the percent passing for each sieve size."""
        return [
            round((total_sample_weight - retain) / total_sample_weight * 100, 2)
            for retain in cumulative_retain
        ]


    def retrieve_and_process_gradation(self, sample_id):
        """
        Retrieves gradation data for the given sample from the database,
        processes it, and updates the results.
        """
        try:
            cursor = self.db_connection.cursor()
    
            cursor.execute(
                "SELECT GradationTestID, TotalSampleWeight, FinesTotalWeight FROM GradationTest WHERE SampleID = ?",
                (sample_id,)
            )
            test_info = cursor.fetchone()
    
            if not test_info:
                return {"success": False, "message": f"No gradation test found for SampleID {sample_id}"}
    
            # Convert retrieved values to float, handling empty strings or None
            test_id = test_info[0]
            total_sample_weight = self.clean_and_convert(test_info[1])
            fines_total_weight = self.clean_and_convert(test_info[2])
    
            if total_sample_weight is None or fines_total_weight is None:
                return {"success": False, "message": "Invalid TotalSampleWeight or FinesTotalWeight in database."}
    
            # Retrieve the associated sieve data
            cursor.execute(
                "SELECT SieveID, WeightRetained FROM GradationResults WHERE GradationTestID = ?",
                (test_id,)
            )
            sieve_data = cursor.fetchall()
    
            if not sieve_data:
                return {"success": False, "message": f"No sieve data found for GradationTestID {test_id}"}
    
            # Process the sieve data
            sieve_order = [{"SieveID": sieve[0]} for sieve in sieve_data]
            weights_retained = [self.clean_and_convert(sieve[1]) for sieve in sieve_data]
    
            # Check if all weights are valid
            if any(weight is None for weight in weights_retained):
                return {"success": False, "message": "Invalid WeightRetained in sieve data."}

            tolerance = .5  # Adjusted tolerance
            weights_sum = sum(weights_retained)
            calculated_total = weights_sum + fines_total_weight
            difference = abs(calculated_total - total_sample_weight)
            
            if difference > tolerance:
                print(f"Debug Info: SampleID {sample_id}")
                print(f"Total Sample Weight: {total_sample_weight}")
                print(f"Sum of Weights Retained: {weights_sum}")
                print(f"Fines Total Weight: {fines_total_weight}")
                print(f"Calculated Total: {calculated_total}")
                print(f"Difference: {difference}")
                return {
                    "success": False,
                    "message": (
                        f"Weights do not sum up to the total sample weight. "
                        f"Calculated: {calculated_total}, Expected: {total_sample_weight}, Difference: {difference}."
                    )
                }
    
            # Calculate cumulative retain and percent passing
            cumulative_retain = self.calculate_cumulative_retain(weights_retained)
            percent_passing = self.calculate_percent_passing(cumulative_retain, total_sample_weight)
    
            # Update the results back into the database
            self.store_gradation_results(test_id, sieve_order, weights_retained, cumulative_retain, percent_passing)
            return {"success": True, "message": f"Gradation test for SampleID {sample_id} processed successfully."}
        except sqlite3.Error as e:
            print(f"Database error: {e}")
            return {"success": False, "message": f"Database error: {e}"}
        except ValueError as e:
            print(f"Value conversion error: {e}")
            return {"success": False, "message": f"Value error: {e}"}
    
    def clean_and_convert(self, value):
        """
        Cleans and converts a value to float. Returns None if the value is invalid.
        """
        if value is None or value == '':
            return 1  # Default value for empty or None entries
        try:
            return float(value)
        except ValueError:
            return None



    def store_gradation_results(self, test_id, sieve_order, weights_retained, cumulative_retain, percent_passing):
        """Updates gradation results in the database."""
        cursor = self.db_connection.cursor()

        try:
            for i, sieve in enumerate(sieve_order):
                query = """
                INSERT INTO GradationResults 
                (GradationTestID, SieveID, WeightRetained, CumulativeRetain, PercentPassing)
                VALUES (?, ?, ?, ?, ?)
                ON CONFLICT(GradationTestID, SieveID) DO UPDATE SET
                    WeightRetained = excluded.WeightRetained,
                    CumulativeRetain = excluded.CumulativeRetain,
                    PercentPassing = excluded.PercentPassing
                """
                values = (
                    test_id,
                    sieve['SieveID'],
                    weights_retained[i],
                    cumulative_retain[i],
                    percent_passing[i]
                )
                cursor.execute(query, values)

            self.db_connection.commit()
            print("Gradation results updated successfully.")
        except sqlite3.Error as e:
            print(f"Error storing gradation results: {e}")
            self.db_connection.rollback()


In [4]:
if __name__ == "__main__":
    db_path = "Soil_framework.sqlite"
    connection = sqlite3.connect(db_path)

    # Process a specific sample by its SampleID
    sample_id = 70 # Replace with the desired SampleID
    gradation_module = GradationModule(connection)
    result = gradation_module.retrieve_and_process_gradation(sample_id)
    print(result)

    connection.close()


Debug Info: SampleID 70
Total Sample Weight: 2071.3
Sum of Weights Retained: 63.4
Fines Total Weight: 148.7
Calculated Total: 212.1
Difference: 1859.2000000000003
{'success': False, 'message': 'Weights do not sum up to the total sample weight. Calculated: 212.1, Expected: 2071.3, Difference: 1859.2000000000003.'}


In [ ]:
### Why do the weights need to add up to thr total smaple weight, can't find an example where previous is true.